In [60]:
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
path = "/home/allanz/data/datasets/v3.1_spatial_grid_multimodal"
dataset = load_from_disk(path)


In [61]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'prompt', 'conversations', 'image'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['text', 'prompt', 'conversations', 'image'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'prompt', 'conversations', 'image'],
        num_rows: 1000
    })
})